In [1]:
# Dependencies
import requests
import json
from config import tmdb_key, omdb_api_key_0_yp, omdb_api_key_1_yp, omdb_api_key_2_mt, omdb_api_key_3_al, omdb_api_key_4_aj
from pprint import pprint
import pandas as pd
from pathlib import Path
import numpy as np


In [2]:
# Set the number of results per page and the total number of desired results
total_result_per_page = 20
total_wanted_results = 2800
# Calculate the total number of pages needed to retrieve the desired results
total_pages = int(total_wanted_results / total_result_per_page) #50
current_page = 1

# Function to fetch data from a specific page using the API
def fetch_page(page_number):
        # Construct the URL with the page number to fetch data from
    url = f"https://api.themoviedb.org/3/discover/movie?&include_video=false&language=en-US&sort_by=popularity.desc&page={page_number}&region=US&with_origin_country=US"
     # Set the required headers for the API request
    headers = {
        "accept": "application/json",
        "Authorization": tmdb_key
    }
    # Make a GET request to the API and retrieve the data in JSON format
    response = requests.get(url, headers=headers).json()
    # Return the results from the response
    return response["results"]

# Initialize an empty list to store all retrieved movies
all_movies = []
#Loop through the range of page numbers to fetch data from each page
for page_number in range(1, total_pages + 1):
    all_movies += fetch_page(page_number)

#Print the total number of movies retrieved
print(len(all_movies))    

#Please uncomment the following if you'd like to check the json outcome
# print(len(response["results"]))
# print(json.dumps(response, indent=4, sort_keys=True))

2800


In [3]:
#Here we created this code to split the all_movies variable call the omdb api 4 times since it has a limitation of 1000 calls per day.

factor = int(len(all_movies) / 4)
#splitted data 
all_movies_splited_0 = all_movies[0:factor] 
all_movies_splited_1 = all_movies[factor:factor * 2]
all_movies_splited_2 = all_movies[factor * 2: factor * 3]
all_movies_splited_3 = all_movies[factor * 3: factor * 4]


In [4]:
# # #collect titles for omdb fetch
# imdb_titles = []
# for imdb_id in all_movies:
#     imdb_titles.append(imdb_id['title'])
# print(len(all_movies))
# imdb_titles 
# # pprint(all_movies[0])

In [4]:
# Function to fetch details of a specific movie using the API, from this details we wanted the finantial information
def fetch_movie_details(movie_id):
    # Construct the URL with the movie ID to fetch details
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    headers = {
        "accept": "application/json",
        "Authorization": tmdb_key
    }
    # Make a GET request to the API and retrieve the movie details in JSON format
    response = requests.get(url, headers=headers).json()
    # Return the movie details from the response
    return response

In [5]:
# Function to fetch omdb data 
def fetch_movie_omdb(movie_title, omdb_api_key):
    url = "http://www.omdbapi.com/?apikey=" + omdb_api_key + "&t="
    # Make a request to the OMDb API for each movie in the list.
    omdb_data = requests.get(url + movie_title).json()
    # Return the omdb data from the response
    return omdb_data

In [6]:
# elements = [all_movies_splited_0,all_movies_splited_1,all_movies_splited_2,all_movies_splited_3]
# api_key_elements = [omdb_api_key_1_yp, omdb_api_key_2_mt, omdb_api_key_3_al, omdb_api_key_4_aj]

elements = [
    {'api_key': omdb_api_key_1_yp, "movies": all_movies_splited_0},
    {'api_key': omdb_api_key_2_mt, "movies": all_movies_splited_1},
    {'api_key': omdb_api_key_3_al, "movies": all_movies_splited_2},
    {'api_key': omdb_api_key_4_aj, "movies": all_movies_splited_3}
]

for element in elements:
    # Iterate through each movie in the list of all movies
    for movie in element['movies']:
        # Fetch details of the current movie using its ID
        movie_details = fetch_movie_details(movie["id"])
        ## Fetch additional movie details(ratings) from the OMDb API using the movie title
        movie_omdb = fetch_movie_omdb(movie['title'], element['api_key'])
        # Update the movie dictionary with the fetched movie details
        movie.update(movie_details)
        movie.update(movie_omdb)

        #Create a key,value pair to determine if movie is financially successful based on revenue and budget comparison
        movie["financial_success"] = movie["revenue"] > movie["budget"]


In [7]:
# Extract rating data one layer up
imdb_rating_name = "Internet Movie Database"
rotten_tomatoes_rating_name = "Rotten Tomatoes"
metacritic_rating_name = 'Metacritic'

for element in elements:
    # Search for ratings
    for movie in element['movies']:
        # Check if "Ratings" key exists
        ratings = movie.get("Ratings", [])
        
        for rating in ratings:
            if imdb_rating_name == rating["Source"]:
                movie["imdb_rating"] = float(rating["Value"].split("/")[0])
            elif rotten_tomatoes_rating_name == rating["Source"]:
                movie["rotten_tomatoes_rating"] = float(rating["Value"].split("%")[0])
            elif metacritic_rating_name == rating["Source"]:
                movie["metacritic_rating"] = float(rating["Value"].split("/")[0])


In [8]:
# delte this
for element in elements:
    print(len(element['movies']))

700
700
700
700


In [11]:
# clear out movies without ratings
test_movies = []
for element in elements:
    movies = element['movies']
    test_movies.extend(list(filter(lambda movie: len(movie['Ratings']) > 0, element['movies'])))

KeyError: 'Ratings'

In [48]:
# delte this
for element in elements:
    print(len(element['movies']))

TypeError: object of type 'filter' has no len()

In [44]:
#Pretty printing our first outcome 
pprint(all_movies_splited_0[1])

{'Actors': 'Ryan Reynolds, Morena Baccarin, T.J. Miller',
 'Awards': '29 wins & 78 nominations',
 'BoxOffice': '$363,070,709',
 'Country': 'United States',
 'DVD': 'N/A',
 'Director': 'Tim Miller',
 'Genre': 'Action, Comedy',
 'Language': 'English',
 'Metascore': '65',
 'Plot': 'A wisecracking mercenary gets experimented on and becomes immortal '
         'yet hideously scarred, and sets out to track down the man who ruined '
         'his looks.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BYzE5MjY1ZDgtMTkyNC00MTMyLThhMjAtZGI5OTE1NzFlZGJjXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_SX300.jpg',
 'Production': 'N/A',
 'Rated': 'R',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '85%'},
             {'Source': 'Metacritic', 'Value': '65/100'}],
 'Released': '12 Feb 2016',
 'Response': 'True',
 'Runtime': '108 min',
 'Title': 'Deadpool',
 'Type': 'movie',
 'Website': 'N/A',
 'Writer': 'Rhett Reese, Paul Wernick',
 'Ye

In [16]:
#Creating a dataframe
def create_dataframe(all_movies_data):
    df = pd.DataFrame(all_movies_data)
    print(df.info())
    return df

all_movies_df_0 = create_dataframe(all_movies_splited_0)
all_movies_df_1 = create_dataframe(all_movies_splited_1)
all_movies_df_2 = create_dataframe(all_movies_splited_2)
all_movies_df_3 = create_dataframe(all_movies_splited_3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 58 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   adult                   700 non-null    bool   
 1   backdrop_path           698 non-null    object 
 2   genre_ids               700 non-null    object 
 3   id                      700 non-null    int64  
 4   original_language       700 non-null    object 
 5   original_title          700 non-null    object 
 6   overview                700 non-null    object 
 7   popularity              700 non-null    float64
 8   poster_path             700 non-null    object 
 9   release_date            700 non-null    object 
 10  title                   700 non-null    object 
 11  video                   700 non-null    bool   
 12  vote_average            700 non-null    float64
 13  vote_count              700 non-null    int64  
 14  belongs_to_collection   382 non-null    ob

In [18]:
# all_movies_df_0.to_csv("movies_dataset_4.csv", index=False)
# all_movies_df_1.to_csv("movies_dataset_1.csv", index=False)
# all_movies_df_2.to_csv("movies_dataset_2.csv", index=False)
# all_movies_df_3.to_csv("movies_dataset_3.csv", index=False)

In [42]:
#Displaying all columns 
print(all_movies_df_3.columns)

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count',
       'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id',
       'origin_country', 'production_companies', 'production_countries',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'Response', 'Error', 'financial_success'],
      dtype='object')


In [20]:
#creating a function to selec specific columns from the dataframe
# Selecting specific columns from the DataFrame
def select_specific_columns(dataframe):
    dataframe = dataframe[["imdb_id", "title","release_date", "Runtime", "Genre","overview","Director", "Actors", "Rated", "imdbRating","imdbVotes", "popularity", "imdb_rating","rotten_tomatoes_rating", "metacritic_rating", "budget", "revenue", "financial_success"]]
    # Drop rows with any missing values
    dataframe.dropna(inplace=True)
    # Display the length of the DataFrame after dropping rows with missing values
    print(len(dataframe))  
    return dataframe


In [43]:
#Putting previous function into action
dataframes = [all_movies_df_0,all_movies_df_1,all_movies_df_2,all_movies_df_3]

for dataframe in dataframes:
    dataframe = select_specific_columns(dataframe)
    display(dataframe.head())


478


,imdb_id,title,release_date,Runtime,Genre,overview,Director,Actors,Rated,imdbRating,imdbVotes,popularity,imdb_rating,rotten_tomatoes_rating,metacritic_rating,budget,revenue,financial_success
1,tt6263850,Deadpool & Wolverine,2024-07-24,108 min,"Action, Comedy",A listless Wade Wilson toils away in civilian ...,Tim Miller,"Ryan Reynolds, Morena Baccarin, T.J. Miller",R,8.0,"1,159,415",2617.082,8.0,85.0,65.0,200000000,1287237740,True
7,tt0094721,Beetlejuice,1988-03-30,92 min,"Comedy, Fantasy",A newly dead New England couple seeks help fro...,Tim Burton,"Alec Baldwin, Geena Davis, Michael Keaton",PG,7.5,"348,874",1048.715,7.5,86.0,70.0,15000000,74849333,True
24,tt27682129,Prey,2024-03-15,100 min,"Action, Adventure, Horror",A young couple is compelled to leave their Chr...,Dan Trachtenberg,"Amber Midthunder, Dakota Beavers, Dane DiLiegro",R,7.1,"233,550",488.732,7.1,94.0,71.0,0,0,False
30,tt0295701,xXx,2002-08-09,124 min,"Action, Adventure, Thriller",Xander Cage is your standard adrenaline junkie...,Rob Cohen,"Vin Diesel, Asia Argento, Marton Csokas",PG-13,5.8,"187,525",369.083,5.8,48.0,48.0,70000000,277448382,True
37,tt4154756,Avengers: Infinity War,2018-04-25,149 min,"Action, Adventure, Sci-Fi",As the Avengers and their allies have continue...,"Anthony Russo, Joe Russo","Robert Downey Jr., Chris Hemsworth, Mark Ruffalo",PG-13,8.4,"1,226,533",270.163,8.4,85.0,68.0,300000000,2052415039,True


KeyError: "['Runtime', 'Genre', 'Director', 'Actors', 'Rated', 'imdbRating', 'imdbVotes', 'imdb_rating', 'rotten_tomatoes_rating', 'metacritic_rating'] not in index"

In [110]:
#splitting some columns 
def splitting_columns(dataframe):
    dataframe[['star_1','star_2','star_3']] = dataframe['Actors'].str.split(',', n=2, expand=True)
    dataframe[['genre_1','genre_2','genre_3']] = dataframe['Genre'].str.split(',', n=2, expand=True)
    dataframe[['director_1','director_2']] = dataframe['Director'].str.split(',', n=1, expand=True)
    dataframe.drop(columns=['Actors', 'Genre', 'Director'], inplace=True)
    return(dataframe)

In [124]:
#Putting previous function into action
all_movies_df_0 = splitting_columns(all_movies_df_0)
all_movies_df_1 = splitting_columns(all_movies_df_1)
all_movies_df_2 = splitting_columns(all_movies_df_2)
all_movies_df_3 = splitting_columns(all_movies_df_3)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,...,BoxOffice,Production,Website,Response,financial_success,imdb_rating,rotten_tomatoes_rating,metacritic_rating,Error,totalSeasons
0,False,/jBUukLTCmXL79itbmK5JtQvImYS.jpg,"[16, 35, 10751]",846214,en,"The Good, the Bart, and the Loki",Loki is banished from Asgard once again and mu...,37.909,/rtMdtzywcAGOrF6t8fbxJBqpdcq.jpg,2021-07-07,...,N/A,N/A,N/A,True,False,6.1,NaN,NaN,NaN,NaN
1,False,/iWKqVHUqWILnvIGN6n6w5zWaJhs.jpg,"[10770, 10749]",1329918,en,Love on the Danube: Love Song,A daughter traveling with her widowed mother a...,37.901,/cDwclk7CiT1ALNz0oF8zdObCBBx.jpg,2024-09-10,...,N/A,N/A,N/A,True,False,NaN,NaN,NaN,NaN,NaN
2,False,/zt6wgtlV6hXDr9tvi3uDJLoHrja.jpg,"[80, 53]",136835,en,No Good Deed,"Terri is a devoted wife and mother of two, liv...",37.883,/wcDLqCS70LsPcovieuQrb311Qr1.jpg,2014-09-10,...,"$52,543,632",N/A,N/A,True,True,5.6,12.0,26.0,NaN,NaN
3,False,/hbvCZ12HiC6Dx4mbgeUr7aHBIPP.jpg,"[16, 10751, 12, 14]",13285,en,Barbie Fairytopia: Mermaidia,"In this animated follow-up to Fairytopia, Elin...",37.874,/igsqS7rpnkAT92Iqo5sYwTs1KWo.jpg,2006-03-14,...,N/A,N/A,N/A,True,False,6.2,NaN,NaN,NaN,NaN
4,False,/zMjI3HqcpOQVcIy9poJQ6vdd2Ds.jpg,"[35, 27, 10402]",974558,en,Destroy All Neighbors,"William Brown, a neurotic, self-absorbed music...",37.872,/jIpjUy6ACDk8Fc8GkynLcIAPP49.jpg,2024-04-26,...,N/A,N/A,N/A,True,False,5.3,72.0,NaN,NaN,NaN


In [17]:
# Rename the following columns to keep the same format
#Putting previous function into action
dataframes = [all_movies_df_0,all_movies_df_1,all_movies_df_2,all_movies_df_3]

for dataframe in dataframes:
    dataframe = dataframe.rename(columns={ "release_date": "release_year","Runtime": "runtime", "Genre": "genre","Director": "director", "Actors": "actors", "Rated": "rated", "imdbRating": "imdb_rating","imdbVotes": "imdb_votes"})
    dataframe.head()

,imdb_id,title,release_year,runtime,genre,overview,director,actors,rated,imdb_rating,...,revenue,financial_success,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,tt8936646,Extraction,2020-04-23,116 min,"Action, Crime, Thriller",A hardened gun-for-hire's latest mission becom...,Sam Hargrave,"Chris Hemsworth, Bryon Lerum, Ryder Lerum",R,6.8,...,0,False,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,None
1,tt2737304,Bird Box,2018-12-13,124 min,"Horror, Mystery, Sci-Fi",Five years after an ominous unseen presence dr...,Susanne Bier,"Sandra Bullock, Trevante Rhodes, John Malkovich",R,6.6,...,0,False,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,None
2,tt0328107,Man on Fire,2004-04-23,146 min,"Action, Crime, Drama",Jaded ex-CIA operative John Creasy reluctantly...,Tony Scott,"Denzel Washington, Christopher Walken, Dakota ...",R,7.7,...,130300000,True,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,None
3,tt3410834,Allegiant,2016-03-09,120 min,"Action, Adventure, Mystery",Beatrice Prior and Tobias Eaton venture into t...,Robert Schwentke,"Shailene Woodley, Theo James, Jeff Daniels",PG-13,5.7,...,179246868,True,Shailene Woodley,Theo James,Jeff Daniels,Action,Adventure,Mystery,Robert Schwentke,None
5,tt0092644,Beverly Hills Cop II,1987-05-18,103 min,"Action, Comedy, Crime",Axel Foley returns to the land of sunshine and...,Tony Scott,"Eddie Murphy, Judge Reinhold, Jürgen Prochnow",R,6.6,...,299965036,True,Eddie Murphy,Judge Reinhold,Jürgen Prochnow,Action,Comedy,Crime,Tony Scott,None


In [18]:
# Format the release_date column to datetime forma
dataframes = [all_movies_df_0,all_movies_df_1,all_movies_df_2,all_movies_df_3]

for dataframe in dataframes:
    dataframe['release_year'] = pd.to_datetime(dataframe['release_year']).dt.year
    dataframe['imdb_rating'] = dataframe['imdb_rating'].astype('float')
    dataframe['rotten_tomatoes_rating'] = dataframe['rotten_tomatoes_rating'].astype('float')
    dataframe['metacritic_rating'] = dataframe['metacritic_rating'].astype('float')
    dataframe['financial_success'] = dataframe['financial_success'].astype('bool')

all_movies_df_2.head()

,imdb_id,title,release_year,runtime,genre,overview,director,actors,rated,imdb_rating,...,revenue,financial_success,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,tt8936646,Extraction,2020,116 min,"Action, Crime, Thriller",A hardened gun-for-hire's latest mission becom...,Sam Hargrave,"Chris Hemsworth, Bryon Lerum, Ryder Lerum",R,6.8,...,0,False,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,None
1,tt2737304,Bird Box,2018,124 min,"Horror, Mystery, Sci-Fi",Five years after an ominous unseen presence dr...,Susanne Bier,"Sandra Bullock, Trevante Rhodes, John Malkovich",R,6.6,...,0,False,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,None
2,tt0328107,Man on Fire,2004,146 min,"Action, Crime, Drama",Jaded ex-CIA operative John Creasy reluctantly...,Tony Scott,"Denzel Washington, Christopher Walken, Dakota ...",R,7.7,...,130300000,True,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,None
3,tt3410834,Allegiant,2016,120 min,"Action, Adventure, Mystery",Beatrice Prior and Tobias Eaton venture into t...,Robert Schwentke,"Shailene Woodley, Theo James, Jeff Daniels",PG-13,5.7,...,179246868,True,Shailene Woodley,Theo James,Jeff Daniels,Action,Adventure,Mystery,Robert Schwentke,None
5,tt0092644,Beverly Hills Cop II,1987,103 min,"Action, Comedy, Crime",Axel Foley returns to the land of sunshine and...,Tony Scott,"Eddie Murphy, Judge Reinhold, Jürgen Prochnow",R,6.6,...,299965036,True,Eddie Murphy,Judge Reinhold,Jürgen Prochnow,Action,Comedy,Crime,Tony Scott,None


In [19]:
all_movies_df_2.dtypes

imdb_id                    object
title                      object
release_year                int32
runtime                    object
genre                      object
overview                   object
director                   object
actors                     object
rated                      object
imdb_rating               float64
imdb_votes                 object
popularity                float64
imdb_rating               float64
rotten_tomatoes_rating    float64
metacritic_rating         float64
budget                      int64
revenue                     int64
financial_success            bool
star_1                     object
star_2                     object
star_3                     object
genre_1                    object
genre_2                    object
genre_3                    object
director_1                 object
director_2                 object
dtype: object

In [6]:
print(all_movies_df_2['financial_success'].dtype)

NameError: name 'all_movies_df_2' is not defined

In [20]:
# outcome
# rules: financial_success == true && imdb_rating > 7 && rotten_tomatoes_rating > 60 && metacritic_rating > 50

# Create a mask for each condition
mask_financial_success = all_movies_df_2['financial_success'] == True
mask_imdb_rating = all_movies_df_2['imdb_rating'] > 6.0
mask_rotten_tomatoes_rating = all_movies_df_2['rotten_tomatoes_rating'] > 60
mask_metacritic_rating = all_movies_df_2['metacritic_rating'] > 60

In [21]:
all_movies_df_2.drop(columns=['financial_success', 'imdb_rating', 'rotten_tomatoes_rating','metacritic_rating'], inplace=True)
all_movies_df_2.head()

,imdb_id,title,release_year,runtime,genre,overview,director,actors,rated,imdb_votes,...,budget,revenue,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,tt8936646,Extraction,2020,116 min,"Action, Crime, Thriller",A hardened gun-for-hire's latest mission becom...,Sam Hargrave,"Chris Hemsworth, Bryon Lerum, Ryder Lerum",R,"267,724",...,65000000,0,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,None
1,tt2737304,Bird Box,2018,124 min,"Horror, Mystery, Sci-Fi",Five years after an ominous unseen presence dr...,Susanne Bier,"Sandra Bullock, Trevante Rhodes, John Malkovich",R,"392,201",...,19800000,0,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,None
2,tt0328107,Man on Fire,2004,146 min,"Action, Crime, Drama",Jaded ex-CIA operative John Creasy reluctantly...,Tony Scott,"Denzel Washington, Christopher Walken, Dakota ...",R,"394,563",...,70000000,130300000,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,None
3,tt3410834,Allegiant,2016,120 min,"Action, Adventure, Mystery",Beatrice Prior and Tobias Eaton venture into t...,Robert Schwentke,"Shailene Woodley, Theo James, Jeff Daniels",PG-13,"136,157",...,110000000,179246868,Shailene Woodley,Theo James,Jeff Daniels,Action,Adventure,Mystery,Robert Schwentke,None
5,tt0092644,Beverly Hills Cop II,1987,103 min,"Action, Comedy, Crime",Axel Foley returns to the land of sunshine and...,Tony Scott,"Eddie Murphy, Judge Reinhold, Jürgen Prochnow",R,"139,487",...,28000000,299965036,Eddie Murphy,Judge Reinhold,Jürgen Prochnow,Action,Comedy,Crime,Tony Scott,None


In [22]:
# Combine the masks using the bitwise AND operator &
all_movies_df_2['outcome'] = mask_financial_success & mask_imdb_rating & mask_rotten_tomatoes_rating & mask_metacritic_rating
# all_movies_df_1['outcome'] = all_movies_df_1['outcome'].astype(int)
# all_movies_df_1['financial_success'] = all_movies_df_1['financial_success'].astype(int)

TypeError: '<' not supported between instances of 'str' and 'int'

In [23]:
#display dataframe
all_movies_df_2.head()

,imdb_id,title,release_year,runtime,genre,overview,director,actors,rated,imdb_votes,...,budget,revenue,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
0,tt8936646,Extraction,2020,116 min,"Action, Crime, Thriller",A hardened gun-for-hire's latest mission becom...,Sam Hargrave,"Chris Hemsworth, Bryon Lerum, Ryder Lerum",R,"267,724",...,65000000,0,Chris Hemsworth,Bryon Lerum,Ryder Lerum,Action,Crime,Thriller,Sam Hargrave,None
1,tt2737304,Bird Box,2018,124 min,"Horror, Mystery, Sci-Fi",Five years after an ominous unseen presence dr...,Susanne Bier,"Sandra Bullock, Trevante Rhodes, John Malkovich",R,"392,201",...,19800000,0,Sandra Bullock,Trevante Rhodes,John Malkovich,Horror,Mystery,Sci-Fi,Susanne Bier,None
2,tt0328107,Man on Fire,2004,146 min,"Action, Crime, Drama",Jaded ex-CIA operative John Creasy reluctantly...,Tony Scott,"Denzel Washington, Christopher Walken, Dakota ...",R,"394,563",...,70000000,130300000,Denzel Washington,Christopher Walken,Dakota Fanning,Action,Crime,Drama,Tony Scott,None
3,tt3410834,Allegiant,2016,120 min,"Action, Adventure, Mystery",Beatrice Prior and Tobias Eaton venture into t...,Robert Schwentke,"Shailene Woodley, Theo James, Jeff Daniels",PG-13,"136,157",...,110000000,179246868,Shailene Woodley,Theo James,Jeff Daniels,Action,Adventure,Mystery,Robert Schwentke,None
5,tt0092644,Beverly Hills Cop II,1987,103 min,"Action, Comedy, Crime",Axel Foley returns to the land of sunshine and...,Tony Scott,"Eddie Murphy, Judge Reinhold, Jürgen Prochnow",R,"139,487",...,28000000,299965036,Eddie Murphy,Judge Reinhold,Jürgen Prochnow,Action,Comedy,Crime,Tony Scott,None


In [24]:
#Saving our DataFrame into 4 different csvs.
# all_movies_df_1.to_csv("movies_dataset_1.csv", index=False)
all_movies_df_2.to_csv("movies_dataset_2.csv", index=False)
# all_movies_df_3.to_csv("movies_dataset_3.csv")
# all_movies_df_4.to_csv("movies_dataset_4.csv")